# Experimenting rec approaches

In [1]:
import polars as pl
import duckdb

In [2]:
CUT_TIMESTAMP = '2017-01-01'

In [3]:
duckdb_conn = duckdb.connect('../data/steam.duckdb', read_only=True)

In [5]:
games_df = duckdb_conn.sql(f"SELECT * FROM game_rolling_features WHERE DATE(game_review_day) <= '{CUT_TIMESTAMP}'").pl()

In [6]:
dim_games_df =duckdb_conn.sql(f"SELECT * FROM dim_games WHERE DATE(game_prerelease_date) <= '{CUT_TIMESTAMP}'").pl()

In [7]:
reviews_df = duckdb_conn.sql(f"SELECT * FROM fact_reviews WHERE DATE(timestamp_created) <= '{CUT_TIMESTAMP}'").pl()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
users_df = duckdb_conn.sql(f"SELECT * FROM dim_users WHERE DATE(first_review_timestamp) <= '{CUT_TIMESTAMP}'").pl()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [29]:
duckdb_conn.close()